# Data Collection

- Songs lyrics corpus

The data used in this project is taken from "Million Song Dataset" project. It consists of:

1. From <https://www.kaggle.com/c/msdchallenge/data>:
   1. train_triplets - Complete listening histories of ~1 Million users. **Used as the training set**.
   3. taste_profile_song_to_tracks - A mapping between profile songs to MSD tracks
   4. kaggle_songs - An indexed list of all songs
   5. kaggle_users - A list of all users

In [1]:
import pandas as pd

train = pd.read_table("./msd/input/train_triplets.txt", header=0, names=['userID', 'songID', 'playsCount'])
train.head()

,userID,songID,playsCount
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBNZDC12A6D4FC103,1


2. From <http://millionsongdataset.com/challenge/#data1>:
   1. year1_test_triplets_visible - Incomplete (half) listening histories of 100,000 users (including plays count for each track). **Used as the test set**
   2. year1_test_triplets_hidden - The other half of litening histories of the 100,000 users. **Used to eavaluate the test predictions**

3. From <http://millionsongdataset.com/pages/getting-dataset/>:
    1. MillionSongSubset - Complete data about 10,000 tracks (randomly selected from the complete Million Song Dataset)

* Note: tracks data is in HDF5 format and is parsed using [hdf5_getters.py](https://github.com/tbertinmahieux/MSongsDB/blob/master/PythonSrc/hdf5_getters.py) 

In [2]:
import pandas as pd
import numpy as np
import msd.hdf5_getters as getter

# For example, the track 'TRADCBA128F92E7161'
h5 = getter.open_h5_file_read('./msd/input/songs_data/TRADCBA128F92E7161.h5')
meta_data_arr = np.array(h5.root.metadata.songs.cols)
analysis_arr = np.array(h5.root.analysis.songs.cols)

analysis_arr


3. From <http://millionsongdataset.com/musixmatch/>:
   1. musiXmatch Dataset - lyrics for many of the MSD songs, in bags-of-words format (for each song, a word-count for a dictionary of the top 5,000 words across the set)

In [4]:
import lyricsgenius

token = 'euu4CMPJm3RJRiAVI2RL_9LDxROSVcrClLw__J_k16Q7hWcfBdCUy0Uq3KPB-0PA'
genius = lyricsgenius.Genius(token, skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
    remove_section_headers=True, sleep_time=0, timeout=4, verbose=False)
# artist = genius.search_artist('Billie Eilish', 10)

lyrics = {}
urls = {}
for song_name, artist_name in songs:
    try:
        urls[song_name] = genius.search_song(song_name, artist_name, get_full_info=False, get_lyrics=False).url
        # lyrics[song_name] = genius.search_song(song_name, artist_name, get_full_info=False).lyrics
    except Exception as e:
        # print(e)
        # break
        print(f'Could not find {song_name} by {artist_name}')


Could not find How Do You Sleep? - Featuring Ludacris by Jesse McCartney


In [7]:
urls

{'Lose Control (feat. Ciara & Fat Man Scoop)': 'https://genius.com/Missy-elliott-lose-control-lyrics',
 'Toxic': 'https://genius.com/Britney-spears-toxic-lyrics',
 'Crazy In Love': 'https://genius.com/Beyonce-crazy-in-love-lyrics',
 'Rock Your Body': 'https://genius.com/Justin-timberlake-rock-your-body-lyrics',
 "It Wasn't Me": 'https://genius.com/Shaggy-it-wasnt-me-lyrics',
 'Yeah!': 'https://genius.com/Usher-yeah-lyrics',
 'My Boo': 'https://genius.com/Usher-and-alicia-keys-my-boo-lyrics',
 'Buttons': 'https://genius.com/The-pussycat-dolls-buttons-lyrics',
 'Say My Name': 'https://genius.com/Destinys-child-say-my-name-lyrics',
 'Hey Ya! - Radio Mix / Club Mix': 'https://genius.com/Ego-trip-magazine-hip-hops-greatest-singles-by-year-lyrics',
 'Promiscuous': 'https://genius.com/Nelly-furtado-promiscuous-lyrics',
 'Right Where You Want Me - Radio Edit Version': 'https://genius.com/Jesse-mccartney-right-where-you-want-me-radio-edit-version-lyrics',
 'Beautiful Soul': 'https://genius.com/